In [1]:
import io
import json
import psycopg2
import pandas as pd
from sqlalchemy import create_engine

In [2]:
def write_to_table(df,schema,table_name,password='1q2w3e4r',host='127.0.0.1',DB='RENT',if_exists='append',port='5432'):
    db_engine = create_engine('postgresql://postgres:{}@{}:{}/{}'.format(password,host,port,DB))
    string_data_io = io.StringIO()
    df.to_csv(string_data_io,sep=',',index=False)
    pd_sql_engine = pd.io.sql.pandasSQL_builder(db_engine)
    table = pd.io.sql.SQLTable(table_name,pd_sql_engine,frame=df,index=False,if_exists=if_exists,schema=schema)
    table.create()
    
    string_data_io.seek(0)
    string_data_io.readline()
    with db_engine.connect() as connection:
        with connection.connection.cursor() as cursor:
            copy_cmd = '''COPY "%s"."%s" FROM STDIN HEADER DELIMITER ',' CSV''' % (schema, table_name)
            cursor.copy_expert(copy_cmd, string_data_io)
        connection.connection.commit()

In [3]:
conn = psycopg2.connect(database='NewHouse',user='postgres',password='1q2w3e4r',host='127.0.0.1',port=5432)

In [4]:
with open("bk_newHouse_map.json", 'r', encoding='utf-8') as fp:
    city_dist = json.loads(fp.read())
city_list = []
for city, districts in city_dist.items():
    city_list.append(city)

In [7]:
count = 0
for i in city_list:
    count += 1
count

593

In [9]:
for city in city_list:
    sql = f"""SELECT id, "城市", "区县", "标题", "销售情况", "分类", "装修", "户型", "单价", "总价", "建面", "最小建面", "最大建面", "容积率", "绿化率", "楼栋总数", "总户数", "建筑面积","地址","标签","开盘时间","物业费","latitude","longitude","抓取月份","抓取年份","数据来源","标题url"
        FROM public."NewHouse_2020" where 城市='{city}';"""
    df_id = pd.read_sql_query(sql, con=conn)
    # print(df_id.shape)
    df_id.drop_duplicates(['标题', '标题url'], inplace=True)
    print(city,df_id.shape)
    write_to_table(df_id, schema='public', table_name='NewHouse_2020',password='1q2w3e4r', host='192.168.88.254',DB='NewHouse' ,port='15432')

安国 (17, 28)
阿拉善盟 (14, 28)
鞍山 (110, 28)
安达 (22, 28)
安庆 (98, 28)
安丘 (34, 28)
安阳 (153, 28)
安陆 (16, 28)
阿坝 (16, 28)
安顺 (33, 28)
安宁 (14, 28)
安康 (72, 28)
阿克苏 (39, 28)
阿拉尔 (14, 28)
北京 (393, 28)
保定 (280, 28)
泊头 (13, 28)
霸州 (39, 28)
包头 (196, 28)
本溪 (71, 28)
北票 (31, 28)
白山 (43, 28)
白城 (26, 28)
北安 (11, 28)
蚌埠 (57, 28)
亳州 (104, 28)
滨州 (160, 28)
北海 (221, 28)
北流 (16, 28)
百色 (72, 28)
保亭市 (15, 28)
巴中 (42, 28)
毕节 (101, 28)
保山 (60, 28)
宝鸡 (139, 28)
白银 (31, 28)
博尔塔拉 (16, 28)
博乐 (19, 28)
承德 (354, 28)
沧州 (50, 28)
长治 (82, 28)
赤峰 (93, 28)
朝阳 (165, 28)
长春 (600, 28)
昌邑 (20, 28)
常州 (400, 28)
常熟 (68, 28)
慈溪 (15, 28)
滁州 (31, 28)
池州 (89, 28)
长葛 (53, 28)
赤壁 (21, 28)
常德 (253, 28)
郴州 (221, 28)
潮州 (71, 28)
岑溪 (16, 28)
崇左 (63, 28)
澄迈市 (39, 28)
成都 (2871, 28)
崇州 (46, 28)
赤水 (11, 28)
楚雄 (23, 28)
昌都 (10, 28)
昌吉 (31, 28)
重庆 (1895, 28)
长沙 (1128, 28)
定州 (66, 28)
大同 (112, 28)
大连 (372, 28)
丹东 (86, 28)
东港 (17, 28)
大石桥 (19, 28)
灯塔 (20, 28)
德惠 (14, 28)
敦化 (19, 28)
大庆 (97, 28)
大兴安岭 (15, 28)
大丰 (21, 28)
东台 (64, 28)
丹阳 (14, 28)
东阳 (2

In [17]:
df_id.to_csv('青岛.csv')